# Ejercicio 3 Capitulo 1
diego.herrerag00@uc.cl

# Primera Parte

## Clases

In [1]:
class Estacion:
    def __init__(self, nombre, ubicacion):
        self.nombre = nombre
        self.ubicacion = ubicacion

    def mostrar_info(self):
        print(f"Estación: {self.nombre}")
        print(f"Ubicación: {self.ubicacion}")

class Pasajero:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    def mostrar_info(self):
        print(f"Pasajero: {self.nombre}")
        print(f"Edad: {self.edad}")

In [2]:
def generar_red_json(estaciones, pasajeros, inicio, fin, nombre_archivo):
    import json, random
    from datetime import timedelta

    viajes = []
    for pasajero in pasajeros:
        hora_actual = inicio
        while hora_actual < fin:
            origen, destino = random.sample(estaciones, 2)
            duracion = timedelta(minutes=random.randint(10, 30)) # Aquí con Timedelta se crea un intervalo de tiempo con la duración que entrege el random. 
            hora_fin = hora_actual + duracion
            if hora_fin > fin:
                break
            viaje = {
                "pasajero": pasajero.nombre,
                "edad": pasajero.edad,
                "origen": origen.nombre,
                "destino": destino.nombre,
                "inicio": hora_actual.strftime("%Y-%m-%d %H:%M"),
                "fin": hora_fin.strftime("%Y-%m-%d %H:%M")
            }
            viajes.append(viaje)
            hora_actual = hora_fin + timedelta(minutes=random.randint(5, 20))

    datos = {
        "estaciones": [{"nombre": e.nombre, "ubicacion": e.ubicacion} for e in estaciones],
        "pasajeros": [{"nombre": p.nombre, "edad": p.edad} for p in pasajeros],
        "viajes": viajes,
        "intervalo": {
            "inicio": inicio.strftime("%Y-%m-%d %H:%M"), #Cada símbolo con % es un código de formato: Y es para año con 4 dígitos, m es para mes con 2 dígitos, 
                                                         #d es para día con 2 dígitos, H es para hora con 2 dígitos, M es para minutos con 2 dígitos.
            "fin": fin.strftime("%Y-%m-%d %H:%M") #strftime transforma un texto a un valor "date". 
        }
    }

    with open(nombre_archivo, "w", encoding="utf-8") as f:
        json.dump(datos, f, ensure_ascii=False, indent=4) #Se le escribe False porque en caso contrario, todos los caracteres que no sean ASCII se guardan como secuencias \uXXXX.
                                                          #Se le da intend=4 para que sea el archivo más legible, con 4 sangrias por nivel, cada nivel es una profundidad del diccionario
                                                          
    print(f"Archivo '{nombre_archivo}' generado correctamente y consistente.")

# Ejemplo de uso
from datetime import datetime # Modulo para representar el momento exacto en el tiempo: 
                              # datetime(year, month, day, hour, minute) = <day> de <month> de <year> a las <hour>:<minute>. Los parametros se entregan como int. 
                              
estaciones = [Estacion("Central", "Santiago Centro"), Estacion("Universidad de Chile", "Santiago Centro"), Estacion("Los Héroes", "Santiago Centro")]
pasajeros = [Pasajero("Ana", 25), Pasajero("Luis", 30), Pasajero("Sofía", 22)]
inicio = datetime(2025, 8, 13, 8, 0) #Aqui ponemos como parametro el año, mes, día, hora y minuto de inicio, el restante, como segundos, microsegundos..quedan en cero por defecto
fin = datetime(2025, 8, 13, 10, 0) # Lo mismo de antes pero para el fin. 
generar_red_json(estaciones, pasajeros, inicio, fin,"red_viajes.json")

Archivo 'red_viajes.json' generado correctamente y consistente.


# Segunda parte

In [3]:
import json
import random
from datetime import datetime

def simulador_red(estaciones, pasajeros, inicio, fin, usar_json=False, archivo_json=None, archivo_salida="simulacion.json"):
    if usar_json and archivo_json:
        with open(archivo_json, "r", encoding="utf-8") as f:
            datos = json.load(f)
    
        
        estaciones = []
        for e in datos['estaciones']:
            estaciones.append(Estacion(e['nombre'], e['ubicacion']))

        pasajeros = []
        for p in datos['pasajeros']:
            pasajeros.append(Pasajero(p['nombre'], p['edad']))

        inicio = datetime.strptime(datos['intervalo']['inicio'], "%Y-%m-%d %H:%M")
        fin = datetime.strptime(datos['intervalo']['fin'], "%Y-%m-%d %H:%M")

    print(f"Simulación iniciada: {inicio} a {fin}")
    eventos = []
    
    minutos_inicio = inicio.hour * 60 + inicio.minute
    minutos_fin = fin.hour * 60 + fin.minute
    fecha = inicio.strftime("%Y-%m-%d")
    for pasajero in pasajeros:
        minutos_actual = minutos_inicio
        while minutos_actual < minutos_fin:
            origen, destino = random.sample(estaciones, 2)
            duracion = random.randint(10, 30) # Aquí se define la duración del viaje en minutos que entrega el random.
            minutos_fin_viaje = minutos_actual + duracion
            if minutos_fin_viaje > minutos_fin:
                break
            hora_inicio = f"{fecha} {minutos_actual//60:02d}:{minutos_actual%60:02d}"
            hora_fin = f"{fecha} {minutos_fin_viaje//60:02d}:{minutos_fin_viaje%60:02d}"
            evento = {
                "pasajero": pasajero.nombre,
                "origen": origen.nombre,
                "destino": destino.nombre,
                "inicio": hora_inicio,
                "fin": hora_fin
            }
            eventos.append(evento)
            minutos_actual = minutos_fin_viaje + random.randint(5, 20)

    resultado = {
        "estaciones": [{"nombre": e.nombre, "ubicacion": e.ubicacion} for e in estaciones],
        "pasajeros": [{"nombre": p.nombre, "edad": p.edad} for p in pasajeros],
        "eventos": eventos,
        "intervalo": {
            "inicio": inicio.strftime("%Y-%m-%d %H:%M"), #Cada símbolo con % es un código de formato: Y es para año con 4 dígitos, m es para mes con 2 dígitos, 
                                                         #d es para día con 2 dígitos, H es para hora con 2 dígitos, M es para minutos con 2 dígitos.
            "fin": fin.strftime("%Y-%m-%d %H:%M") #strftime transforma un texto a un valor "date".
        }
    }

    with open(archivo_salida, "w", encoding="utf-8") as f:
        json.dump(resultado, f, ensure_ascii=False, indent=4) #Se le escribe False porque en caso contrario, todos los caracteres que no sean ASCII se guardan como secuencias \uXXXX.
                                                              #Se le da intend=4 para que sea el archivo más legible, con 4 sangrias por nivel, cada nivel es una profundidad del diccionario
                                                          
    print(f"Simulación finalizada. Archivo '{archivo_salida}' generado.")


# Ejemplo de uso
estaciones = [Estacion("Central", "Santiago Centro"), Estacion("Universidad de Chile", "Santiago Centro"), Estacion("Los Héroes", "Santiago Centro")]
pasajeros = [Pasajero("Ana", 25), Pasajero("Luis", 30), Pasajero("Sofía", 22)]
inicio = datetime(2025, 8, 13, 8, 0) #Aqui ponemos como parametro el año, mes, día, hora y minuto de inicio, el restante, como segundos, microsegundos..quedan en cero por defecto
fin = datetime(2025, 8, 13, 10, 0) # Lo mismo de antes pero para el fin. 
simulador_red(estaciones, pasajeros, inicio, fin)


Simulación iniciada: 2025-08-13 08:00:00 a 2025-08-13 10:00:00
Simulación finalizada. Archivo 'simulacion.json' generado.


# Tercera Parte

In [4]:
import json
from datetime import datetime

def consultas_simulacion(archivo_simulacion):
    with open(archivo_simulacion, "r", encoding="utf-8") as f:
        datos = json.load(f)
    
    if "eventos" in datos:
        eventos = datos["eventos"]
    else:
        eventos = datos["viajes"]


    # Estaciones con mayor tránsito de pasajeros
    transito = {} #Creamos un diccionario "transito" para contar cuántas veces aparece cada estación como origen o destino.
    for evento in eventos:
        transito[evento["origen"]] = transito.get(evento["origen"], 0) + 1 #Esto es clave, dict.get(clave, 0) devuelve el valor actual si existe, 
                                                                            # y en caso de que no exista, devuelve 0. 
        transito[evento["destino"]] = transito.get(evento["destino"], 0) + 1
    estacion_mas_transito = max(transito, key=transito.get)
    cantidad = transito[estacion_mas_transito]
    print(f"Estación con mayor tránsito: {estacion_mas_transito} ({cantidad} pasajeros)")

    # Viaje de mayor duración
    max_duracion = 0
    viaje_largo = None
    for evento in eventos:
        inicio = datetime.strptime(evento["inicio"], "%Y-%m-%d %H:%M") #Cada símbolo con % es un código de formato: Y es para año con 4 dígitos, m es para mes con 2 dígitos, 
                                                                       #d es para día con 2 dígitos, H es para hora con 2 dígitos, M es para minutos con 2 dígitos.
        fin = datetime.strptime(evento["fin"], "%Y-%m-%d %H:%M") #strftime transforma un texto a un valor "date".
        
        duracion = (fin - inicio).total_seconds() / 60 #total_seconds convierte una diferencia timedelta a segundos en float
        if duracion > max_duracion:
            max_duracion = duracion
            viaje_largo = evento
    print(f"Viaje más largo: {viaje_largo['pasajero']} de {viaje_largo['origen']} a {viaje_largo['destino']} ({max_duracion:.1f} minutos)")

    # Aquí vemos los pasajeros con mas viajes
    viajes_por_pasajero = {}
    for evento in eventos:
        nombre = evento["pasajero"]
        viajes_por_pasajero[nombre] = viajes_por_pasajero.get(nombre, 0) + 1
    pasajero_top = max(viajes_por_pasajero, key=viajes_por_pasajero.get)
    viajes_top = viajes_por_pasajero[pasajero_top]
    print(f"Pasajero con más viajes: {pasajero_top} ({viajes_top} viajes)")


consultas_simulacion("simulacion.json")


Estación con mayor tránsito: Central (8 pasajeros)
Viaje más largo: Ana de Los Héroes a Universidad de Chile (30.0 minutos)
Pasajero con más viajes: Ana (4 viajes)


## Test Case

### Test de Clases

In [5]:
import unittest


class TestE3Clases(unittest.TestCase): #Debe heredar de la clase TestCase de la libreria de unittest. 
    def test_estacion_atributos(self):
        est = Estacion("Central", "Santiago Centro")
        self.assertEqual(est.nombre, "Central") #assertEqual(a, b) verifica que a == b. En caso de que se cumpla, nos avisa con un "ok"
                                                #En caso de que no se cumpla, nos avisa con un "FAIL"
        self.assertEqual(est.ubicacion, "Santiago Centro")

    def test_pasajero_atributos(self):
        pas = Pasajero("Ana", 25)
        self.assertEqual(pas.nombre, "Ana")
        self.assertEqual(pas.edad, 25)

    def test_mostrar_info_estacion(self):
        est = Estacion("Universidad de Chile", "Santiago Centro")
        # Para verificar que el método no tenga errores
        est.mostrar_info()

    def test_mostrar_info_pasajero(self):
        pas = Pasajero("Luis", 30)
        # Para verificar que el método no tenga errores
        pas.mostrar_info()

        # Para verificar que el método no tenga errores
        pas.mostrar_info()

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)

test_estacion_atributos (__main__.TestE3Clases.test_estacion_atributos) ... ok
test_mostrar_info_estacion (__main__.TestE3Clases.test_mostrar_info_estacion) ... ok
test_mostrar_info_pasajero (__main__.TestE3Clases.test_mostrar_info_pasajero) ... ok
test_pasajero_atributos (__main__.TestE3Clases.test_pasajero_atributos) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.001s

OK


Estación: Universidad de Chile
Ubicación: Santiago Centro
Pasajero: Luis
Edad: 30
Pasajero: Luis
Edad: 30


In [6]:
from datetime import datetime

class TestE3Funciones(unittest.TestCase):
    def setUp(self):
        self.estaciones = [Estacion("Central", "Santiago Centro"), Estacion("Universidad de Chile", "Santiago Centro")]
        self.pasajeros = [Pasajero("Ana", 25), Pasajero("Luis", 30)]
        self.inicio = datetime(2025, 8, 13, 8, 0)
        self.fin = datetime(2025, 8, 13, 9, 0)

    def test_generar_red_json_crea_archivo(self):
        import os
        nombre_archivo = "test_red_viajes.json"
        generar_red_json(self.estaciones, self.pasajeros, self.inicio, self.fin, nombre_archivo)
        self.assertTrue(os.path.exists(nombre_archivo)) #Esto verifica que una expresión sea verdadera
        os.remove(nombre_archivo) #Se elimina el archivo

    def test_simulador_red_simple_crea_archivo(self):
        import os
        nombre_archivo = "test_simulacion_simple.json"
        simulador_red(self.estaciones, self.pasajeros, self.inicio, self.fin, archivo_salida=nombre_archivo)
        self.assertTrue(os.path.exists(nombre_archivo))
        os.remove(nombre_archivo)

    def test_consultas_simulacion_funciona(self):
        nombre_archivo = "test_simulacion.json"
        generar_red_json(self.estaciones, self.pasajeros, self.inicio, self.fin, nombre_archivo)
        # Solo verificamos que la función no arroje error
        consultas_simulacion(nombre_archivo)
        import os
        os.remove(nombre_archivo)

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)

test_estacion_atributos (__main__.TestE3Clases.test_estacion_atributos) ... ok
test_mostrar_info_estacion (__main__.TestE3Clases.test_mostrar_info_estacion) ... ok
test_mostrar_info_pasajero (__main__.TestE3Clases.test_mostrar_info_pasajero) ... ok
test_pasajero_atributos (__main__.TestE3Clases.test_pasajero_atributos) ... ok
test_consultas_simulacion_funciona (__main__.TestE3Funciones.test_consultas_simulacion_funciona) ... ok
test_generar_red_json_crea_archivo (__main__.TestE3Funciones.test_generar_red_json_crea_archivo) ... ok
test_simulador_red_simple_crea_archivo (__main__.TestE3Funciones.test_simulador_red_simple_crea_archivo) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.005s

OK


Estación: Universidad de Chile
Ubicación: Santiago Centro
Pasajero: Luis
Edad: 30
Pasajero: Luis
Edad: 30
Archivo 'test_simulacion.json' generado correctamente y consistente.
Estación con mayor tránsito: Central (5 pasajeros)
Viaje más largo: Luis de Universidad de Chile a Central (23.0 minutos)
Pasajero con más viajes: Ana (3 viajes)
Archivo 'test_red_viajes.json' generado correctamente y consistente.
Simulación iniciada: 2025-08-13 08:00:00 a 2025-08-13 09:00:00
Simulación finalizada. Archivo 'test_simulacion_simple.json' generado.
